In [2]:
from ber_metrics import BEREstimator
import numpy as np
import pickle
import pandas as pd

## Adult Income

In [6]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/adult_income/processed_data.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
adult_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(adult_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(adult_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(adult_estimator.nn_bound(leafsize=1000)))


Bhattacharrya upper bound: 1.0878476909979809e-39
Mahalanobis upper bound: 0.2200983607931948


RecursionError: maximum recursion depth exceeded while calling a Python object

## COMPAS Recidivism

In [4]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_violent.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound(X_test, y_test)))

Bhattacharrya upper bound: 0.02596945557312596
Mahalanobis upper bound: 0.18821591060670279


In [ ]:
X_train, y_train, X_test, y_test = pickle.load(open('Data/compas/processed_arrest.pkl', 'rb'))
X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
violent_estimator = BEREstimator(X_train, y_train)
print("Bhattacharrya upper bound: {}".format(violent_estimator.bhattacharyya_bound()))
print("Mahalanobis upper bound: {}".format(violent_estimator.mahalanobis_bound()))
print("Nearest Neighbor upper bound: {}".format(violent_estimator.nn_bound(X_test, y_test)))